In [33]:
import xml.etree.ElementTree as et, os
import collections.abc       as abc
import pandas                as pd
import qgrid

from os.path                 import join
from framenet.util           import flatten, curry, flatmap, take
from framenet.builder        import build
from glob                    import iglob
from collections             import OrderedDict
from typing                  import Callable, List, Sequence
from multipledispatch        import dispatch
from framenet.util           import cata
from pprint                  import pformat, pprint
from framenet.ecg.generation import (root_for, base_for, Tree, 
                                     T, unstack, 
                                     EtTree, TestTree as TT)

# For Pandas
pd.options.display.max_rows    = 99
pd.options.display.max_columns = 199


tagify = lambda tag, item: '%sID' % tag if item == 'ID' else item

# The ICSI namespace we use below, as in 'fn:sentence'
fn = {'fn': 'http://framenet.icsi.berkeley.edu'}

lu10  = root_for['lu']('lu10')
asets = lu10.findall('.//fn:sentence', fn)

# Test: unstacking one XML file

In [34]:
# The ICSI namespace we use below, as in 'fn:sentence'
fn = {'fn': 'http://framenet.icsi.berkeley.edu'}

# This creates a loader for file 'lu10.xml' from the 'lu' folder 
lu10           = root_for['lu']('lu10')

# Find all <sentence ...> tags
lu10_sentences = lu10.findall('.//fn:sentence', fn)

def unstack_one(elem):
    "Unstack the whole XML subtree at `elem`."
    return [dict(flatten(vs)) for vs in unstack(2, EtTree(elem))]

lu10_df = pd.DataFrame(flatmap(unstack_one, asets))
# qgrid.nbinstall(overwrite=True)

In [35]:
qgrid.show_grid(lu10_df, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100})

## Now, let's define a function that does the above:

Concretely, given an XML element, return a list of dictionaries. We need the list of dictionaries to finally create the `DataFrame` object.

In [36]:
unstack_all = lambda elements: flatmap(unstack_one, elements)

## Next:

1. Gather all the LU files by creating loaders for each one;
1. Extract the <sentence...> elements;
1. Apply the above to each one;
1. There's no 4!!!

In [40]:
from glob    import iglob 
from os.path import join

lu_roots = map(root_for['lu'], iglob(join(base_for['lu'], '*.xml')))
lu_sents = flatten(r.findall('.//fn:sentence', fn) for r in lu_roots)

In [41]:
# Warning: this will take a long time. Only need to do this once!

LU_PICKLE = 'lu.pkl'

if os.access(LU_PICKLE, os.R_OK):
    # Read lu_df back in
    lu_df = pd.read_pickle(LU_PICKLE)
else:
    # Save to a file in the current directory
    lu_df = pd.DataFrame(list(unstack_all(lu_sents)))
    %time lu_df.to_pickle(LU_PICKLE)

CPU times: user 5.8 s, sys: 2.08 s, total: 7.88 s
Wall time: 8.62 s


## Let's do the same for the full text annotations

In [42]:
ft_roots = map(root_for['fulltext'], iglob(join(base_for['fulltext'], '*.xml')))
ft_sents = flatten(r.findall('.//fn:sentence', fn) for r in ft_roots)

In [43]:
# Same as above
FT_PICKLE = 'fulltext.pkl'

if os.access(FT_PICKLE, os.R_OK):
    # Read lu_df back in
    ft_df = pd.read_pickle(FT_PICKLE)
else:
    # Save to a file in the current directory
    ft_df = pd.DataFrame(list(unstack_all(ft_sents)))
    %time ft_df.to_pickle(FT_PICKLE)

CPU times: user 481 ms, sys: 196 ms, total: 677 ms
Wall time: 714 ms


## TODO

1. function :: Frame -> Core FEs
1. Use them to select records

In [44]:
# Read FN data in 
fn, fnb = build()

In [45]:
Cause_motion   = fn.get_frame('Cause_motion')
cm_core_fe_ids = [int(e.ID) for e in Cause_motion.elements if e.coreType == 'Core']
cm_nc_fe_ids   = [int(e.ID) for e in Cause_motion.elements if e.coreType != 'Core']

In [46]:
len(Cause_motion.lexicalUnits)

38

In [47]:
cm_core_fe_ids

[228, 229, 230, 231, 232, 234, 4991, 5532, 7031]

In [48]:
# LUs for Cause_motion
fnb.build_lus_for_frame('Cause_motion', fn)

In [49]:
# Pick annotation IDs for Cause_motion
cm_aset_ids  = [int(ann.ID) for ann in Cause_motion.annotations]

In [50]:
cm_df = lu_df.loc[
    lu_df['annotationSet.ID'].isin(cm_aset_ids) 
    & ~ lu_df['label.feID'].isin(cm_nc_fe_ids)
]
len(cm_df)

10689

In [51]:
qgrid.show_grid(cm_df, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100})

### Note: Let's verify none the records in cm_df involve noncore FEs!

In [52]:
assert len(cm_df[cm_df['label.feID'].isin(cm_nc_fe_ids)]) == 0

In [53]:
cm_ext_df = pd.concat([cm_df, 
                       pd.get_dummies(cm_df.loc[:, ['label.name']], prefix_sep=':')], 
                      axis=1)

In [58]:
pd.options.display.max_rows=500

cols1 = ['sentence.ID', 'label.start', 
         'label.itype', 'label.name', 'label.feID', 'layer.name']

cols2 = ['sentence.ID', 'label.start', 
         'label.name', 'layer.name']

cols3 = ['sentence.ID', 'label.start', 'label.name', 
         'label.name:Ext', 'label.name:Dep', 'label.name:Obj']

cols4 = ['sentence.ID', 'label.start', 
         'label.name:Ext', 'label.name:Dep', 'label.name:Obj']

In [78]:
gs = cm_ext_df.loc[:, cols2].groupby(cols2[:3])
gs.first()

layer.name
sentence.ID label.start label.name           
580494      0.0         Agent              FE
                        Ext                GF
                        NP                 PT
            2.0         Target         Target
            13.0        NP                 PT
                        Obj                GF
                        Theme              FE
            29.0        Dep                GF
                        PP                 PT
                        Path               FE
            50.0        Dep                GF
                        Goal               FE
                        PP                 PT
580498      0.0         Agent              FE
                        Ext                GF
                        Metaphor         Sent
                        NP                 PT
            29.0        Target         Target
            38.0        NP                 PT
                        Obj                GF
                        Theme              FE
            42.0        Dep                GF
                        Goal               FE
                        PP                 PT
580507      0.0         Metaphor         Sent
            83.0        Agent              FE
                        Ant             Other
                        Ext                GF
                        NP                 PT
            143.0       Agent              FE
                        Ext                GF
                        NP                 PT
                        Rel             Other
            148.0       Target         Target
            159.0       NP                 PT
                        Obj                GF
                        Theme              FE
            164.0       Dep                GF
                        Goal               FE
                        PP                 PT
580517      0.0         Metaphor         Sent
            23.0        Agent              FE
                        Ant             Other
                        Ext                GF
                        NP                 PT
            43.0        Agent              FE
                        Ext                GF
                        NP                 PT
                        Rel             Other
            55.0        Target         Target
            64.0        NP                 PT
                        Obj                GF
                        Theme              FE
            72.0        Dep                GF
                        Goal               FE
                        PP                 PT
580527      0.0         Ext                GF
                        NP                 PT
                        Theme              FE
            30.0        Target         Target
            41.0        Dep                GF
                        PP                 PT
                        Source             FE
580529      0.0         Metaphor         Sent
            28.0        Ext                GF
                        NP                 PT
                        Theme              FE
            35.0        Target         Target
            46.0        Dep                GF
                        Goal               FE
                        PPing              PT
580533      55.0        Ext                GF
                        NP                 PT
                        Theme              FE
            116.0       Target         Target
            127.0       Dep                GF
                        Goal               FE
                        PP                 PT
580537      0.0         Ext                GF
                        NP                 PT
                        Theme              FE
            57.0        Target         Target
            68.0        Dep                GF
                        Goal               FE
                        PP                 PT
            105.0       Dep      